In [3]:
import pandas as pd
df_ana = pd.read_csv("../dados/df_preparado.csv")

In [4]:
df_ana.columns

Index(['id', 'id_unique', 'numero_sei', 'ano_documento', 'assinaturas',
       'descricao_tipo_documento', 'descricao_tipo_processo',
       'numero_processo', 'id_unidade', 'sigla_unidade', 'descricao_unidade',
       'data_documento', 'setor_economico', 'setor_economico_classe',
       'setor_economico_divisao', 'setor_economico_grupo',
       'setor_economico_secao', 'setor_economico_sub_classe',
       'partes_processos_like', 'partes_processos', 'decisao_sg',
       'link_documento', 'mercado_relevante', 'documentos_relacionados',
       'descricao_especificacao', 'data_processo', '_version_', 'corpo_texto',
       'decisao_tribunal', 'conteudo', 'diferenca_dias'],
      dtype='object')

In [5]:
df_ana["decisao_tribunal"].value_counts()

decisao_tribunal
condenacao      559
arquivamento    172
vazio           163
Name: count, dtype: int64

In [6]:
df_ana20 = df_ana.head(20)

#### Regex

In [7]:
import re
def limpar_texto_cade(texto):
    
    # 1. Remove cabeçalhos e rodapés institucionais
    texto = re.sub(r'(?i)(timbre|ministério|conselho|endereço|telefone|cep|www\.).*?\n', '', texto)
    
    # 2. Remove URLs
    texto = re.sub(r'https?://\S+', '', texto)

    # 3. Remove expressões genéricas
    texto = re.sub(r'\[ACESSO RESTRITO\]', '', texto, flags=re.IGNORECASE)
    texto = re.sub(r'Brasília\s*,?\s*DF\s*-?\s*data da assinatura eletrônica.*?\n', '', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\(sistema eletrônico\)', '', texto, flags=re.IGNORECASE)

    # 4. Remove notas de rodapé
    texto = re.sub(r'\[\d+\]', '', texto)

    # 5. Remove trechos longos de leis (mais agressivo)
    texto = re.sub(r'(Art\.?|artigo)\s+\d+.*?(Lei|Código|Decreto).*?(\n|$)', '(referência legal suprimida)', texto, flags=re.DOTALL)

    # 6. Remove números de página
    texto = re.sub(r'\n?\s*\d{1,3}\s*\n', '\n', texto)

    # 7. Remove datas formais
    texto = re.sub(r'\b\d{2}/\d{2}/\d{4}\b', '', texto)
    texto = re.sub(r'\b\d{4}-\d{2}-\d{2}\b', '', texto)

    # 8. Remove detalhes processuais irrelevantes
    texto = re.sub(r'SEI/CADE - \d+ - .*?\n', '', texto)
    texto = re.sub(r'Representante:.*?\n', '', texto)
    texto = re.sub(r'Representados?:.*?\n', '', texto)
    texto = re.sub(r'Relator\(a\)?:.*?\n', '', texto)

    # 9. Remove tabelas e quadros
    texto = re.sub(r'(\+\-+\+.*?\+\-+\+)', '', texto, flags=re.DOTALL)

    texto = re.sub(r'\|.*?\|', '', texto)
    
    # 10. Mantém apenas as seções críticas
    secoes_chave = re.findall(
        r'(?:CONCLUSÕES DA SUPERINTENDÊNCIA-GERAL|CONCLUSÕES DA PROCURADORIA|DO MÉRITO|DISPOSITIVO|voto pelo).*?(?:\n\n|$)',
        texto, 
        flags=re.DOTALL | re.IGNORECASE
    )
    
    texto = ' '.join(secoes_chave) if secoes_chave else texto

    # 11. Limpeza final
    texto = re.sub(r'\s{2,}', ' ', texto)
    texto = re.sub(r'\n{2,}', '\n', texto)
    texto = re.sub(r'(?i)(voto|decisão|condenação|arquivamento|multa)', '\n\\1', texto)  # Melhora estrutura
    
    return texto.strip()

In [9]:
import os, json
import pandas as pd
from dotenv import load_dotenv
from tqdm.auto import tqdm
import google.generativeai as genai

load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

# Configura o Gemini
genai.configure(api_key=API_KEY)
modelo = genai.GenerativeModel(
    "gemini-1.5-flash",
    generation_config={"temperature": 0.3}  
)

/home/gabriel/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# def extrair_informacoes_juridicas(texto) :
#     prompt = f"""
#     Analise exclusivamente o DISPOSITIVO FINAL e CONCLUSÕES do documento jurídico do CADE abaixo.
#     Ignore discussões processuais, evidências detalhadas ou fundamentação extensa.

#     **Instruções específicas:**
#     1. Para 'decisao_tribunal': considerar apenas os termos finais ("voto pelo arquivamento", "voto pela condenação")
#     2. Para 'seguiu_nota_tecnica': verificar se o relator menciona explicitamente seguir ou divergir da nota técnica
#     3. Para valores de multa: extrair apenas valores explícitos no dispositivo final
#     4. Quando não encontrar informação clara, usar null

#     Documento:
#     \"\"\"{texto}\"\"\"

#     Devolva APENAS JSON com os seguintes campos:
#     {{
#       "decisao_tribunal": "Condenação" | "Arquivamento" | null,
#       "seguiu_nota_tecnica": true | false | null,
#       "tipo_infracao_concorrencial": string | null,
#       "multa": true | false,
#       "tipo_de_multa": "valor_fixo" | "percentual" | "ambos" | null,
#       "valor_multa_reais": float | null,
#       "percentual_faturamento": float | null
#     }}
#     """.strip()

#     resposta = modelo.generate_content(prompt)
    
#     # Isola o JSON da resposta
#     try:
#         conteudo = resposta.text
#         bloco_json = conteudo[conteudo.find('{'):conteudo.rfind('}')+1]
#         return json.loads(bloco_json)
#     except Exception as e:
#         print(f"[ERRO ao parsear JSON]: {e}")
#         print("Resposta bruta do Gemini:\n", resposta.text)
#         return {
#             "decisao_tribunal": None,
#             "seguiu_nota_tecnica": None,
#             "tipo_infracao_concorrencial": None,
#             "multa": None,
#             "tipo_de_multa": None,
#             "valor_multa_reais": None,
#             "percentual_faturamento": None
#         }

In [ ]:
import re
import json
from datetime import datetime

def extrair_informacoes_juridicas(texto):
    # 1. Pré-processamento - Isolar seções críticas
    dispositivo = extrair_secao(texto, r"DISPOSITIVO\s*FINAL", r"CONCLUS[OÕ]ES|Assinatura|$")
    conclusoes = extrair_secao(texto, r"CONCLUS[OÕ]ES", r"DISPOSITIVO|V\.?\s*E\.?\s*M\.?\s*I\.?|$")
    texto_relevante = (dispositivo + "\n" + conclusoes).strip()

    # 2. Decisão do Tribunal
    decisao = classificar_decisao(texto_relevante)
    
    # 3. Relação com Nota Técnica
    seguimento_tecnica = detectar_seguimento_tecnica(texto_relevante)
    
    # 4. Informações sobre Multa
    resultado_multa = extrair_multa(texto_relevante)

    # 5. Tipo de Infração
    infracao = identificar_infracao(texto_relevante)

    return {
        "decisao_tribunal": decisao,
        "seguiu_nota_tecnica": seguimento_tecnica,
        "tipo_infracao_concorrencial": infracao,
        "multa": resultado_multa["multa"],
        "tipo_de_multa": resultado_multa["tipo"],
        "valor_multa_reais": resultado_multa["valor_reais"],
        "percentual_faturamento": resultado_multa["percentual"]
    }

#--- Funções Auxiliares ---
def extrair_secao(texto, inicio_regex, fim_regex):
    try:
        inicio_match = re.search(inicio_regex, texto, re.IGNORECASE)
        if not inicio_match:
            return ""
        
        inicio_idx = inicio_match.start()
        resto_texto = texto[inicio_idx:]
        
        fim_match = re.search(fim_regex, resto_texto, re.IGNORECASE)
        fim_idx = fim_match.start() if fim_match else len(resto_texto)
        
        return resto_texto[:fim_idx]
    except Exception:
        return ""

def classificar_decisao(texto):
    condenacao_padroes = [
        r"voto\s+pela\s+condena[cç][aã]o",
        r"impondo-se\s+multa",
        r"condeno",
        r"aplica[cç][aã]o\s+de\s+multa"
    ]
    
    arquivamento_padroes = [
        r"voto\s+pelo\s+arquivamento",
        r"extin[cç][aã]o\s+sem\s+resolu[cç][aã]o\s+do\s+m[eé]rito",
        r"insufici[eê]ncia\s+probat[oó]ria"
    ]

    for padrao in condenacao_padroes:
        if re.search(padrao, texto, re.IGNORECASE):
            return "Condenação"
            
    for padrao in arquivamento_padroes:
        if re.search(padrao, texto, re.IGNORECASE):
            return "Arquivamento"
            
    return None

def detectar_seguimento_tecnica(texto):
    seguimento_padroes = [
        r"em\s+sintonia\s+com\s+(a\s+)?nota\s+t[eé]cnica",
        r"acolh(endo|e)\s+(integralmente\s+)?(o\s+)?parecer\s+t[eé]cnico",
        r"nos\s+termos\s+do\s+laudo",
        r"seguindo\s+as\s+recomenda[cç][oõ]es\s+t[eé]cnicas"
    ]
    
    divergencia_padroes = [
        r"diverg(indo|ência|ente)\s+(da|do)\s+an[aá]lise\s+t[eé]cnica",
        r"em\s+desacordo\s+com\s+(a\s+)?nota\s+t[eé]cnica",
        r"não\s+acompanha\s+(as\s+)?conclus[oõ]es\s+t[eé]cnicas",
        r"contrariamente\s+ao\s+parecer\s+t[eé]cnico"
    ]

    for padrao in seguimento_padroes:
        if re.search(padrao, texto, re.IGNORECASE):
            return True
            
    for padrao in divergencia_padroes:
        if re.search(padrao, texto, re.IGNORECASE):
            return False
            
    return None

def extrair_multa(texto):
    # Padrões monetários aprimorados
    padrao_valor = re.compile(
        r"R\$\s*([\d.,]+)\s*(milh[aã]o|milh[õo]es|mil|bi|lh[õo]es|bilh[aã]o|bilh[õo]es)?",
        re.IGNORECASE
    )
    
    # Padrões percentuais
    padrao_percentual = re.compile(
        r"(\d{1,3}(?:\.\d{3})*(?:,\d+)?%|\d+,\d+\s*%\s+do\s+faturamento)",
        re.IGNORECASE
    )

    # Encontra valores
    valores = []
    for match in padrao_valor.finditer(texto):
        valor_bruto = match.group(1)
        escala = match.group(2).lower() if match.group(2) else ""
        valor = normalizar_valor(valor_bruto, escala)
        if valor:
            valores.append(valor)
    
    # Encontra percentuais
    percentuais = []
    for match in padrao_percentual.finditer(texto):
        percentual = normalizar_percentual(match.group())
        if percentual is not None:
            percentuais.append(percentual)

    # Determinação do tipo de multa
    tem_valor = len(valores) > 0
    tem_percentual = len(percentuais) > 0

    # Verificação de ausência de multa
    sem_multa_padroes = [
        r"sem\s+imposi[cç][aã]o\s+de\s+multa",
        r"isento\s+de\s+multa",
        r"apenas\s+san[cç][oõ]es\s+comportamentais"
    ]
    
    for padrao in sem_multa_padroes:
        if re.search(padrao, texto, re.IGNORECASE):
            return {
                "multa": False,
                "tipo": None,
                "valor_reais": None,
                "percentual": None
            }

    return {
        "multa": tem_valor or tem_percentual,
        "tipo": "ambos" if tem_valor and tem_percentual 
                else "valor_fixo" if tem_valor 
                else "percentual" if tem_percentual 
                else None,
        "valor_reais": max(valores) if valores else None,
        "percentual": max(percentuais) if percentuais else None
    }

def normalizar_valor(valor_str, escala=""):
    try:
        # Converter diferentes formatos numéricos
        valor_str = valor_str.replace(".", "").replace(",", ".")
        
        # Remover caracteres não numéricos
        valor_limpo = re.sub(r"[^\d.]", "", valor_str)
        valor_base = float(valor_limpo)
        
        # Aplicar fatores de escala
        fatores = {
            "mil": 1e3,
            "milhao": 1e6, "milhão": 1e6, "milhoes": 1e6, "milhões": 1e6,
            "bilhao": 1e9, "bilhão": 1e9, "bilhoes": 1e9, "bilhões": 1e9
        }
        
        for termo, fator in fatores.items():
            if termo in escala.lower():
                return valor_base * fator
        
        return valor_base
    except Exception:
        return None

def normalizar_percentual(percent_str):
    try:
        # Extrair número do percentual
        num_match = re.search(r"(\d+[\d.,]*)", percent_str.replace(".", "").replace(",", "."))
        if num_match:
            return float(num_match.group(1))
        return None
    except Exception:
        return None

def identificar_infracao(texto):
    # Mapeamento de infração com expressões regulares
    infracoes_map = {
        r"cartel|conluio|acordo\s+horizontal": "Cartel",
        r"abuso\s+de\s+poder\s+de\s+mercado|posi[cç][aã]o\s+dominante": "Abuso de Poder de Mercado",
        r"venda\s+casada|condicionamento": "Venda Casada",
        r"condutas\s+verticais|restri[cç][oõ]es\s+verticais": "Condutas Verticais",
        r"pr[aá]tica\s+comercial\s+restritiva": "Prática Comercial Restritiva"
    }
    
    # Tentar encontrar infração específica
    for padrao, infracao in infracoes_map.items():
        if re.search(padrao, texto, re.IGNORECASE):
            return infracao
    
    # Fallback para menção genérica
    infracao_match = re.search(
        r"infra[cç][aã]o\s+[aà]\s+(art\.\s*)?\d+[^\n\.;]+",
        texto, 
        re.IGNORECASE
    )
    
    if infracao_match:
        return infracao_match.group(0).strip()
        
    return None

In [11]:
def processar_linha(texto):
    #texto_limpo = limpar_texto_cade(texto)
    return extrair_informacoes_juridicas(texto)

In [12]:
df_ana20['resultado'] = df_ana20['conteudo'].apply(processar_linha)

# Expande o dicionário da coluna 'resultado' em colunas separadas
df_expandido = pd.concat([df_ana20, df_ana20['resultado'].apply(pd.Series)], axis=1)

# Remove a coluna intermediária (opcional)
df_expandido.drop(columns=['resultado'], inplace=True)

# Mostra o DataFrame resultante
df_expandido

/tmp/ipykernel_44141/153724095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ana20['resultado'] = df_ana20['conteudo'].apply(processar_linha)


,id,id_unique,numero_sei,ano_documento,assinaturas,descricao_tipo_documento,descricao_tipo_processo,numero_processo,id_unidade,sigla_unidade,...,decisao_tribunal,conteudo,diferenca_dias,decisao_tribunal,seguiu_nota_tecnica,tipo_infracao_concorrencial,multa,tipo_de_multa,valor_multa_reais,percentual_faturamento
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,jurisprudencia__1390607,1390607,2024,Outras,Voto Processo Administrativo,Processo Administrativo,08700.004558/2019-05,110000967.0,GAB1,...,vazio,SEI/CADE - 1390607 - Voto Processo Administrat...,1714,Condenação,False,cartel,True,valor_fixo,324116.21,NaN
1,b41e8f1f-449e-4d28-a574-18f365d32b6f,jurisprudencia__1384594,1384594,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.004974/2015-71,110000969.0,GAB3,...,condenacao,SEI/CADE - 1384594 - Voto Processo Administrat...,3276,Condenação,True,imposição de tabela de preços mínimos,True,valor_fixo,100000.00,NaN
2,adec625d-4f5e-4514-b938-a295fdc1e8a4,jurisprudencia__1384685,1384685,2024,Outras,Voto,Processo Administrativo,08700.005915/2022-40,110001021.0,DIAP,...,arquivamento,SEI/CADE - 1384685 - Voto\n\n\nProcesso nº 087...,640,Arquivamento,True,cartel no mercado de revenda de combustíveis,False,None,NaN,NaN
3,c8499708-5778-45d9-8914-9ab97b77c1fd,jurisprudencia__1377990,1377990,2024,Alexandre Cordeiro Macedo,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000955.0,GAB-PRES,...,condenacao,SEI/CADE - 1377990 - Voto Processo Administrat...,2709,Condenação,None,cartel em licitação,True,valor_fixo,NaN,NaN
4,29758e3d-7047-4c5a-8ac6-b6815c67ec3d,jurisprudencia__1377023,1377023,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000969.0,GAB3,...,condenacao,SEI/CADE - 1377023 - Voto Processo Administrat...,2709,Arquivamento,True,cartel,True,percentual,NaN,NaN
5,15187e9b-d1e4-4c9d-97e2-721755174c1c,jurisprudencia__1377847,1377847,2024,Outras,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000972.0,GAB6,...,condenacao,SEI/CADE - 1377847 - Voto Processo Administrat...,2709,Arquivamento,None,None,False,None,NaN,NaN
6,2aabf3bb-af87-47ed-b09b-0726d04cdf5f,jurisprudencia__1377767,1377767,2024,Outras,Voto Processo Administrativo,Processo Administrativo,08700.007776/2016-41,110000967.0,GAB1,...,condenacao,SEI/CADE - 1377767 - Voto Processo Administrat...,2709,Condenação,True,cartel em licitação,True,valor_fixo,14845237.43,15.0
7,0d278d0f-5ab8-44cc-9ec6-cde5c399d9f9,jurisprudencia__1375782,1375782,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.003266/2022-42,110000969.0,GAB3,...,arquivamento,SEI/CADE - 1375782 - Voto Processo Administrat...,706,Arquivamento,True,cartel,False,None,NaN,NaN
8,84fe3a4b-a1b4-4536-957e-2aa5c372527f,jurisprudencia__1362271,1362271,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08700.003699/2017-31,110000969.0,GAB3,...,vazio,SEI/CADE - 1362271 - Voto Processo Administrat...,2464,Condenação,False,cartel de licitação,True,valor_fixo,86517771.33,NaN
9,5c7e5985-7239-4ab1-9662-ddc610b2c48d,jurisprudencia__1358814,1358814,2024,Gustavo Augusto Freitas de Lima,Voto Processo Administrativo,Processo Administrativo,08012.006043/2008-37,110000969.0,GAB3,...,condenacao,SEI/CADE - 1358814 - Voto Processo Administrat...,5763,NÃO CONHECIMENTO,None,cartel,True,percentual,NaN,20.0
